In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import joblib
import time

In [14]:
# Load the dataset
file_path = '../datasets/train_set.csv'
train_df = pd.read_csv(file_path)
train_df = train_df.drop(columns=['Assembly Code', 'Assembly Description', 'Type Name'])

# Define preprocessing for categorical features
categorical_features = ['Family', 'SubFamily', 'ObjectGroup', 'ObjectName', 'Description', 'Type Comments', 'Structural Material', 'Material']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

# Create the model pipeline with a Random Forest Classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))  # Random state for reproducibility
])

# Split the data into training and testing sets
X = train_df.drop('Category', axis=1)
y = train_df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Get best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Refit the model on the entire training set with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Save the trained model to a file
model_filename = 'random_forest_model_cross.pkl'
joblib.dump(best_model, model_filename)

# Predict and evaluate with best model on test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

# Optional: Print total runtime
total_time = grid_search.refit_time_  # This gives the total time for fitting the best model during the grid search
print(f"Total runtime: {total_time} seconds")


Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\xiang\miniconda3\envs\tf\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters found:  {}
Best cross-validation score:  0.9202803101303569
                                                    precision    recall  f1-score   support

                         Adjustable Louvred Window       0.82      0.90      0.86        10
                              Architectural Column       1.00      1.00      1.00       107
                              Cast In-Situ RC Wall       1.00      0.75      0.86        12
                                  Compartment Wall       0.00      0.00      0.00         1
                                      Curtain Wall       1.00      1.00      1.00        65
                                          Dry Wall       0.00      0.00      0.00         2
                     Fire Alarm Fixture and Device       1.00      1.00      1.00         6
                              Fixed Louvred Window       1.00      1.00      1.00         4
                                Fixed Panel Window       1.00      1.00      1.00         3
   

c:\Users\xiang\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xiang\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xiang\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
